In [1]:
import csv 


import csv
import re
import calendar

from tqdm import tqdm


def save_csv(path,columns,tpl):
        
    with open(path,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(columns)
        csv_out.writerow(tpl)
        
def update_csv(path,tpl):
    with open(path,'a') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(tpl)
        


def process_address(txt):
    x = re.findall(r"[\w]+|[&,/#-]+", txt)
    #x = list(filter(lambda a: a != '', x))
    x =  list(filter(lambda a: a != 'amp', x))
    out = ' '.join(x)
    return out;

def past_price(past_price_d,address,year):
    
    try: return past_price_d[address][year];
    except: return float('nan');

vic_df = pd.read_csv('processed.csv');


#month_to_num_d = {month: index for index, month in enumerate(calendar.month_name) if month};

vic_df['address_processed'] = vic_df['address'].apply(lambda x : process_address(x))
vicy_df = vic_df.groupby(["address_processed","year","type_1","code","suburb","car","bath","bed"]).mean().reset_index()

vic_essential_df = vicy_df[['address_processed','year','rent_pw']];

duplicate_d = {}

total = len(vic_essential_df);

with tqdm(total=total) as pbar:
    

    for index, row in vic_essential_df.iterrows():
        address = row["address_processed"]
        year = row["year"];
        rent = row["rent_pw"];
    
        try:duplicate_d[address][year] = rent;
        except: 
            duplicate_d[address] ={};
            duplicate_d[address][year] = rent;
            
        pbar.update(1)
        
key_l = list(duplicate_d.keys())
total = len(key_l);

for i in tqdm(range(total)):
    
    if len(duplicate_d[key_l[i]]) ==1:
        
        del duplicate_d[key_l[i]]

key_l = list(duplicate_d.keys())
total = len(key_l);

past_price_d = {}

for i in tqdm(range(total)):

    address = key_l[i];
    year_l = list(duplicate_d[address].keys());
    
    for i, j in zip(year_l[:-1], year_l[1:]):
        
        if j-i == 3:
            
            
            try: past_price_d[address][j] = duplicate_d[address][i];
            except:
                past_price_d[address] ={};
                past_price_d[address][j] = duplicate_d[address][i];
                
#key_l = list(past_price_d.keys());

vicy_df["rent_yminthree"] = vicy_df.apply(lambda row : past_price(past_price_d,row.address_processed,row.year),axis=1)
vicy_past_df = vicy_df[vicy_df["rent_yminthree"].notna()]

100%|██████████| 554719/554719 [00:00<00:00, 719660.02it/s]


NameError: name 'rate_growth_d' is not defined

In [11]:
vicy_past_df.reset_index().drop(columns = ['index']).to_csv("rentminthree.csv",index=False)

In [11]:
geocode_df = pd.read_csv('geocode.csv');
coded_df = vic_df.join(geocode_df.set_index('address'), on='address');
fullycoded_df = coded_df[~coded_df['full_address'].isnull()];
len(fullycoded_df)

7328